# Valuing European-style Swaptions with QL Example

We value a European swaption using Black's model and try to replicate a QL example at http://khandrikacm.blogspot.com/2014/03/european-style-interest-rate-swaption.html

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.products.rates import *
from financepy.utils import *
from financepy.market.curves import *
from financepy.models.FinModelBlack import FinModelBlack

####################################################################
# FINANCEPY BETA Version 0.185 - This build:  24 Oct 2020 at 21:27 #
#      This software is distributed FREE & WITHOUT ANY WARRANTY    #
# For info and disclaimer - https://github.com/domokane/FinancePy  #
#      Send any bug reports or comments to quant@financepy.com     #
####################################################################



## Building a Ibor Discount Curve

In [3]:
valuation_date = Date(28, 2, 2014)
settlement_date = Date(4, 3, 2014)

We have a vector of dates and zero rates

In [4]:
depoDCCType = DayCountTypes.THIRTY_E_360_ISDA
depos = []

depo = FinIborDeposit(settlement_date, "1W", 0.0023, depoDCCType); depos.append(depo)
depo = FinIborDeposit(settlement_date, "1M", 0.0023, depoDCCType); depos.append(depo)
depo = FinIborDeposit(settlement_date, "3M", 0.0023, depoDCCType); depos.append(depo)
depo = FinIborDeposit(settlement_date, "6M", 0.0023, depoDCCType); depos.append(depo)

# No convexity correction provided so I omit interest rate futures

swaps = []
swapType = FinSwapTypes.PAY
fixedDCCType = DayCountTypes.ACT_365F
fixedFreqType = FrequencyTypes.SEMI_ANNUAL

swap = FinIborSwap(settlement_date, "3Y", swapType, 0.00790, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinIborSwap(settlement_date, "4Y", swapType, 0.01200, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinIborSwap(settlement_date, "5Y", swapType, 0.01570, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinIborSwap(settlement_date, "6Y", swapType, 0.01865, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinIborSwap(settlement_date, "7Y", swapType, 0.02160, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinIborSwap(settlement_date, "8Y", swapType, 0.02350, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinIborSwap(settlement_date, "9Y", swapType, 0.02540, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinIborSwap(settlement_date, "10Y", swapType, 0.0273, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinIborSwap(settlement_date, "15Y", swapType, 0.0297, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinIborSwap(settlement_date, "20Y", swapType, 0.0316, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinIborSwap(settlement_date, "25Y", swapType, 0.0335, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinIborSwap(settlement_date, "30Y", swapType, 0.0354, fixedFreqType, fixedDCCType); swaps.append(swap)

libor_curve = FinIborSingleCurve(valuation_date, depos, [], swaps)

Inserting synthetic deposit


In [5]:
exercise_date = settlement_date.addTenor("5Y")
swapMaturityDate = exercise_date.addTenor("5Y")

swapFixedCoupon = 0.040852
swapFixedFrequencyType = FrequencyTypes.SEMI_ANNUAL
swapFixedDayCountType = DayCountTypes.THIRTY_E_360_ISDA

swapFloatFrequencyType = FrequencyTypes.QUARTERLY
swapFloatDayCountType = DayCountTypes.ACT_360

swapNotional = ONE_MILLION
swapType = FinSwapTypes.PAY 
calendar_type = CalendarTypes.TARGET
bus_day_adjust_type = BusDayAdjustTypes.MODIFIED_FOLLOWING
date_gen_rule_type = DateGenRuleTypes.BACKWARD

In [6]:
swaption = FinIborSwaption(settlement_date,
                            exercise_date,
                            swapMaturityDate,
                            swapType,
                            swapFixedCoupon,
                            swapFixedFrequencyType,
                            swapFixedDayCountType, 
                            swapNotional, 
                            swapFloatFrequencyType,
                            swapFloatDayCountType,
                            calendar_type, 
                            bus_day_adjust_type,
                            date_gen_rule_type)

In [7]:
print(swaption)

OBJECT TYPE: FinIborSwaption
SETTLEMENT DATE: TUE 04 MAR 2014
EXERCISE DATE: MON 04 MAR 2019
SWAP TYPE: FinSwapTypes.PAY
SWAP MATURITY DATE: MON 04 MAR 2024
SWAP NOTIONAL: 1000000
FIXED COUPON: 4.0852
FIXED FREQUENCY: FrequencyTypes.SEMI_ANNUAL
FIXED DAY COUNT: DayCountTypes.THIRTY_E_360_ISDA
FLOAT FREQUENCY: FrequencyTypes.QUARTERLY
FLOAT DAY COUNT: DayCountTypes.ACT_360



In [8]:
model = FinModelBlack(0.1533)

In [9]:
swaption.value(settlement_date, libor_curve, model)

23163.13292186464

This is close to a value of 23162 using QL

## Internals

In [10]:
print(swaption)

OBJECT TYPE: FinIborSwaption
SETTLEMENT DATE: TUE 04 MAR 2014
EXERCISE DATE: MON 04 MAR 2019
SWAP TYPE: FinSwapTypes.PAY
SWAP MATURITY DATE: MON 04 MAR 2024
SWAP NOTIONAL: 1000000
FIXED COUPON: 4.0852
FIXED FREQUENCY: FrequencyTypes.SEMI_ANNUAL
FIXED DAY COUNT: DayCountTypes.THIRTY_E_360_ISDA
FLOAT FREQUENCY: FrequencyTypes.QUARTERLY
FLOAT DAY COUNT: DayCountTypes.ACT_360
PV01: 4.16168532945034
FWD SWAP RATE: 4.086035922736006
FWD DF TO EXPIRY: 0.9237697293451264


We can see that the forward swap rate almost equals the fixed coupon. The underlying swap is close to being ATM forward.

In [11]:
swaption.print_swap_fixed_leg()

START DATE: MON 04 MAR 2019
MATURITY DATE: MON 04 MAR 2024
COUPON (%): 4.0852
FIXED LEG FREQUENCY: FrequencyTypes.SEMI_ANNUAL
FIXED LEG DAY COUNT: DayCountTypes.THIRTY_E_360_ISDA
VALUATION DATE TUE 04 MAR 2014
PAYMENT_DATE     YEAR_FRAC        FLOW         DF         DF*FLOW       CUM_PV
TUE 04 MAR 2014          -            -   1.00000000            -            -
WED 04 SEP 2019  0.5000000     20426.00   0.90872318     18561.58     18561.58
WED 04 MAR 2020  0.5000000     20426.00   0.89262979     18232.86     36794.44
FRI 04 SEP 2020  0.5000000     20426.00   0.87518398     17876.51     54670.94
THU 04 MAR 2021  0.5000000     20426.00   0.85693776     17503.81     72174.75
MON 06 SEP 2021  0.5055556     20652.96   0.84085474     17366.14     89540.89
FRI 04 MAR 2022  0.4944444     20199.04   0.82477877     16659.74    106200.63
MON 05 SEP 2022  0.5027778     20539.48   0.80762580     16588.21    122788.85
MON 06 MAR 2023  0.5027778     20539.48   0.79020647     16230.43    139019.27


In [14]:
swaption.print_swap_float_leg()

START DATE: MON 04 MAR 2019
MATURITY DATE: MON 04 MAR 2024
SPREAD COUPON (%): 0.0
FLOAT LEG FREQUENCY: FrequencyTypes.QUARTERLY
FLOAT LEG DAY COUNT: DayCountTypes.ACT_360
VALUATION DATE TUE 04 MAR 2014
Floating Flows not calculated.


Copyright (c) 2020 Dominic O'Kane